In [1]:
import pandas as pd

data = pd.read_csv('~/Downloads/ml-price-prediction-dataset.csv', sep=';', low_memory=False, encoding='latin')

In [2]:
data

,restaurant_id,restaurant_name,testing,restaurant_description,product_id,variation_id,menu_category,product_name,product_description,price,...,longitude,chain_id,restaurant_active,city_id,dietary_characteristics,cuisine_characteristics,taste_characteristics,preparation_style_characteristics,dish_type_characteristics,ingredients
0,453,fit kitchen eat fit,0,gesundes fast food,35039,35037,fit pizza,pizza margherita,tomatensauce und fettarmer kaese,6.0,...,8.684631,NaN,1,9,['lakto vegetarisch'],"['gesundes essen', 'italienisch']","['mild', 'natuerlich']",['gebacken'],['pizza'],"['kaese', 'mehl', 'tomaten']"
1,453,fit kitchen eat fit,0,gesundes fast food,35040,35038,fit pizza,"pizza salami, putenschinken","fettarme salami, putenschinken und fettarmer k...",8.4,...,8.684631,NaN,1,9,['lakto vegetarisch'],"['gesundes essen', 'italienisch']",['natuerlich'],['gebacken'],['pizza'],"['kaese', 'mehl', 'salami', 'schinken', 'tomat..."
2,453,fit kitchen eat fit,0,gesundes fast food,35043,35041,fit pizza,pizza veggie + rucola,"pilze, zwiebeln, rucola und fettarmer kaese",8.4,...,8.684631,NaN,1,9,"['lakto vegetarisch', 'ohne rindfleisch', 'ohn...","['gesundes essen', 'italienisch']","['mild', 'natuerlich']",['gebacken'],['pizza'],"['kaese', 'mehl', 'pilz', 'rucola', 'tomatensa..."
3,453,fit kitchen eat fit,0,gesundes fast food,35045,35043,fit pizza,pizza sucuk jalapenos,"fettarme knoblauchwurst, jalapenos und fettarm...",8.4,...,8.684631,NaN,1,9,NaN,['italienisch'],['natuerlich'],['gebacken'],['pizza'],['jalapeno']
4,453,fit kitchen eat fit,0,gesundes fast food,35046,35044,fit pizza,pizza haehnchenbrust,"magere haehnchenbruststreifen, pilze, zwiebeln...",8.9,...,8.684631,NaN,1,9,['halal'],"['gesundes essen', 'italienisch']",['natuerlich'],['gebacken'],['pizza'],"['haehnchenbrustfilet', 'kaese', 'mehl', 'toma..."
5,453,fit kitchen eat fit,0,gesundes fast food,35051,35049,fit burger,classic burger,"mageres rindfleisch, eisbergsalat, tomaten, zw...",5.9,...,8.684631,NaN,1,9,['ohne schweinefleisch'],"['amerikanisch', 'gesundes essen']","['mild', 'natuerlich']",['gebraten'],['burger'],"['eisbergsalat', 'gurke', 'rindfleisch', 'toma..."
6,453,fit kitchen eat fit,0,gesundes fast food,35052,35050,fit burger,cheese burger,"mageres rindfleisch, low fat kaese, eisbergsal...",6.4,...,8.684631,NaN,1,9,['ohne schweinefleisch'],"['amerikanisch', 'gesundes essen']","['mild', 'natuerlich']",['gebraten'],['burger'],"['eisbergsalat', 'gurke', 'kaese', 'rindfleisc..."
7,453,fit kitchen eat fit,0,gesundes fast food,35053,35051,fit burger,chili cheese burger,"mageres rindfleisch, salat, jalapenos und fit ...",7.4,...,8.684631,NaN,1,9,['ohne schweinefleisch'],"['amerikanisch', 'gesundes essen']","['leicht scharf', 'natuerlich']",['gebraten'],['burger'],"['jalapeno', 'rindfleisch', 'salat']"
8,453,fit kitchen eat fit,0,gesundes fast food,35059,35057,fit beilagen,fit pommes,mit heissluft frittierte pommes,2.9,...,8.684631,NaN,1,9,['vegan'],"['amerikanisch', 'gesundes essen']",['natuerlich'],"['frittiert', 'gebacken']",['pommes'],['kartoffel']
9,453,fit kitchen eat fit,0,gesundes fast food,35060,35058,fit beilagen,fit coleslaw,fettarmer krautsalat,2.9,...,8.684631,NaN,1,9,NaN,['international'],['natuerlich'],['fusion mixed'],['extra'],['krautsalat']


In [3]:
from ast import literal_eval
import numpy as np

def multi_hot_encode(col):
    d = data[col]
    catset = set()
    for cats in d.values:
        if isinstance(cats, float):  
            # n.b. may make sense to try imputing averages for missing values instead of zeroing
            continue
        cats = literal_eval(cats)
        for cat in cats:
            catset.add(cat)
    
    indexing = {}
    for i, cat in enumerate(catset):
        indexing[cat] = i
    
    feats = np.zeros((len(d), len(indexing)), dtype='uint8')
    for i, elem in enumerate(d):
        if isinstance(elem, float):
            continue
        elem = literal_eval(elem)
        for e in elem:
            feats[i, indexing[e]] = 1
    
    return feats

categorical_cols = ['dietary_characteristics', 'cuisine_characteristics', 'taste_characteristics', 
                    'preparation_style_characteristics', 'dish_type_characteristics', 'ingredients']
all_multi_cat_feats = list(map(multi_hot_encode, categorical_cols))
for feat in all_multi_cat_feats:
    print(feat.shape)
    
all_multi_cat_feats_stacked = np.hstack(all_multi_cat_feats)
print(all_multi_cat_feats_stacked.shape)

assert(all_multi_cat_feats_stacked.shape[1] == sum(map(lambda x: x.shape[1], all_multi_cat_feats)))

(65499, 11)
(65499, 105)
(65499, 10)
(65499, 22)
(65499, 62)
(65499, 761)
(65499, 971)


In [8]:
import spacy
import de_core_news_sm
nlp = de_core_news_sm.load()

text_cols = ['menu_category','product_name', 'product_description']
data_filled = data.fillna('')
texts = data_filled['menu_category'] + ' ' + data_filled['product_name'] + ' ' + data_filled['product_description']
texts = list(map(nlp, texts))
print(texts[0])

fit pizza pizza margherita tomatensauce und fettarmer kaese


In [61]:
# texts = list(map(str, texts))
print(texts[0])

from sklearn.feature_extraction.text import TfidfVectorizer

def generate_tfidf_features(texts):
    vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_features=1000)
    # n.b. experiment with max_features here. we don't want to have too many features relative to dataset size though.
    # could also be worthwhile to go with maximum here and apply pca after to reduce final dimensionality
    vectorized = vectorizer.fit_transform(texts)
    return vectorized

text_feats = generate_tfidf_features(texts)
print(text_feats.shape)
print(all_multi_cat_feats_stacked.shape)

from scipy import sparse

full_feats = sparse.hstack([all_multi_cat_feats_stacked, text_feats])
print(full_feats.shape)

Pizza Margherita Tomatensauce und fettarmer Käse
(65499, 1000)
(65499, 974)
(65499, 1974)


In [ ]:
# Definitely try adding a one hot encoding of the restaurant ID. This is most likely one of the strongest price
# indicators I would say intuitively. But one has to be careful due to the small sample sizes per restaurant.
# As I'm currently not having too much time to look into this I won't include it in the experiments here.
# Maybe something like compressing the one hot encoding of restaurant ID via PCA into few dimensions is helpful.
# Adding 10k features as a one hot encoded restaurant ID will introduce too many features given the sample size of 65k
# I'd say.

In [23]:
targets = data['price']
print(min(targets))
print(max(targets))
s = 0
for i, d in enumerate(data['price']):
    if d == 0.0:
        s += 1
        print(data.ix[i])
        
print(s)
        
# There are some items with price 0. These are usually not reflecting dishes it seems at first glance.
# May make sense to exclude those from the whole process, but needs closer analysis to be sure.

0.0
550.0
restaurant_id                                         138
restaurant_name                                  Chopstix
testing                                                 0
restaurant_description               Der Geschmack Asiens
product_id                                          47227
variation_id                                        47225
menu_category                                     Starter
product_name                                      Sup Tom
product_description                        Garnelen Suppe
price                                                   0
postcode                                     10178 Berlin
latitude                                          52.5228
longitude                                         13.4096
chain_id                                              NaN
restaurant_active                                       0
city_id                                                 7
dietary_characteristics                               NaN
cuis

In [62]:
from sklearn.linear_model import Lasso # Should try other regression models and evaluate them properly via cv.
from sklearn.model_selection import train_test_split

# n.b. do a proper cross validation here!
from numpy.random import permutation
randomized = permutation(range(len(data)))
print(randomized)

full_feats = full_feats.todense()
print(full_feats.shape, targets.shape)

max_ind = int(0.1 * len(randomized))
X_test = full_feats[:max_ind]
y_test = targets[:max_ind]
X_train = full_feats[max_ind:]
y_train = targets[max_ind:]

print(X_test.shape, y_test.shape, X_train.shape, y_train.shape)

model = Lasso(alpha=0.9, tol=1e-9)
model.fit(X_train, y_train)
model.score(X_test, y_test)

[60598 19124 19241 ..., 11745 36754 55230]
(65499, 1974) (65499,)
(6549, 1974) (6549,) (58950, 1974) (58950,)


-0.015411508410851036

In [55]:
for i in range(100):
    print(model.predict(X_train[i]))
    print(y_train[i])


[ 19.36668949]
9.5
[ 8.25757028]
4.6
[ 8.25757028]
5.1
[ 8.25757028]
4.8
[ 8.25757028]
4.2
[ 8.25757028]
4.6
[ 8.25757028]
4.8
[ 8.25757028]
4.6
[ 8.25757028]
4.6
[ 8.25757028]
13.6
[ 8.25757028]
13.6
[ 8.25757028]
13.6
[ 8.25757028]
23.2
[ 8.25757028]
22.2
[ 8.25757028]
13.6
[ 8.25757028]
12.9
[ 8.25757028]
12.9
[ 8.25757028]
2.9
[ 8.25757028]
2.9
[ 8.25757028]
2.9
[ 8.25757028]
2.2
[ 8.25757028]
2.9
[ 8.25757028]
13.6
[ 8.25757028]
12.9
[ 8.25757028]
3.9
[ 6.79257431]
3.6
[ 6.80992312]
3.6
[ 15.70618985]
4.9
[ 6.58679099]
2.3
[ 14.95025388]
4.9
[ 18.12817632]
26.0
[ 20.62656012]
23.2
[ 17.00723963]
20.8
[ 20.80905783]
20.3
[ 18.30740649]
24.8
[ 17.20525924]
24.8
[ 17.18842473]
27.5
[ 8.25757028]
10.0
[ 8.25757028]
14.5
[ 8.25757028]
20.5
[ 8.25757028]
6.0
[ 8.25757028]
5.5
[ 8.25757028]
5.5
[ 8.25757028]
6.9
[ 8.25757028]
5.5
[ 8.25757028]
5.0
[ 8.25757028]
5.5
[ 8.25757028]
7.5
[ 8.25757028]
5.5
[ 8.25757028]
5.5
[ 8.25757028]
7.5
[ 8.25757028]
7.5
[ 8.25757028]
7.9
[ 8.25757028]
6.

In [63]:
errs = 0.0
for i in range(X_test.shape[0]):
    real = y_test[i]
    act = model.predict(X_test[i])
    errs += abs(real-act)

errs /= X_test.shape[0]
print(errs)

# Lasso seems to not work very well...

[ 4.72498264]


In [64]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

errs = 0.0
for i in range(X_test.shape[0]):
    real = y_test[i]
    act = model.predict(X_test[i])
    errs += abs(real-act)

errs /= X_test.shape[0]
print(errs)

# Better but still not great...
# I assume it would help to remove irrelevant features and play around with hyperparameters more.

[ 3.61942483]
